### DN1 - Naloga 1
Avtor: Urh Primožič

Najprej razdelimo podatke na testno in učno množico.

In [2]:
import pandas as pd #cvs importing
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

RANDOM_SEED = 420
df = pd.read_csv('podatki_DN1_1.csv', header=0)
columns = df.columns[1:] #1. stolpec je le dekoracijski
df = df[columns]
X = df[['x1', 'x2', 'x3', 'x4', 'x5']]
y = df['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=2/3, random_state=RANDOM_SEED)
df

,x1,x2,x3,x4,x5,y
0,10.493885,-0.359581,0.254364,0.593046,0.432593,0.023722
1,12.481717,-0.796263,1.193631,-1.098599,0.721224,0.014009
2,17.110862,0.696601,0.289725,0.392631,0.202218,0.012581
3,10.305360,0.410720,0.030711,0.430333,0.843228,0.031607
4,7.848507,0.675717,2.141359,-0.398808,0.370072,0.058590
...,...,...,...,...,...,...
564,15.717126,-0.871626,-1.131575,0.288426,0.720055,0.008563
565,2.379988,0.554697,-1.157613,1.055736,0.323168,0.525161
566,11.586536,0.426375,-0.358417,-0.064733,0.849829,0.025199
567,0.641669,-0.056897,-0.270050,0.754823,0.591815,1.670335


Model najbližjih sosedov.

In [3]:
knn3 = KNeighborsRegressor(n_neighbors=3)
knn3.fit(X_train, y_train)
mse3 = mean_squared_error(y_test, knn3.predict(X_test))
print('Napaka knn modela za k=3 je ', mse3)

Napaka knn modela za k=3 je  0.012423075075045954


Iskanje optimalnega števila sosedov s pomočjo bootstrapinga

In [12]:
from sklearn.utils import resample
from tqdm import tqdm
k_range = list(range(1,10))
def boot(X, y,model, n_iterations):
    '''
    Z bootstrapingom oceni napako modela.
    Parameters
    ------------
    - X, y - podatki
    - model - razred z metodama fit() and predict()
    - n_iterations - št_vzorcev (=št_iteracij)

    Vrne
    -------
    Povprečno napako na vseh vzorcih (to je manj optimistična opcija)
    '''
    vzorci = [resample(X, y) for i in range(n_iterations)]
    mse = []
    for i in range(n_iterations):
        # naučiš se na enem vzorcu
        X_train = vzorci[i][0]
        y_train = vzorci[i][1]
        model.fit(X_train, y_train)
        # potestiraš na ostalih
        # i-ta napaka je mse na ostalih vzorcih
        # ker so vsi vzorci enako veliki (veliki so |X|)
        # dobimo enako, če seštejemo vse napake na vzorcih in delimo s št_ ostalih vzorccev
        err = 0
        for j in range(n_iterations):
            if j == i:
                continue
            X_test = vzorci[j][0]
            y_test = vzorci[j][1]
            err += mean_squared_error(y_test, model.predict(X_test))
        err /= n_iterations -1
        mse.append(err)
    return  np.mean(mse)


def knn_boot(X, y, n_iterations, param_space):
    '''
    Poišče najbolši k za knn z bootstrapingom

    Parametri
    ----------
    - X, y - podatki
    - param_space - vrednosti k-jev, ki jih testiramo
    Vrne
    ---------
    dictionary with keys
        'opt' - optimalen parameter
        'mse'
    '''
    napaka = float('inf')
    opt = -1
    # če ni tqdm-ja:
    #for k in param_space:
    for k in tqdm(param_space, total=len(param_space)):
        tmp_napaka = boot(X, y, KNeighborsRegressor(n_neighbors=k), n_iterations)
        if tmp_napaka < napaka:
            napaka = tmp_napaka
            opt = k
    return {'opt': opt, 'mse' : napaka}
        
optimalen = knn_boot(X_train, y_train, 100, k_range)  
k = optimalen['opt']
min_napaka = optimalen['mse']
print(f'Optimalen k je {k}, z napako  {min_napaka} ')




100%|██████████| 9/9 [04:21<00:00, 29.01s/it]

Optimalen k je 1, z napako  0.013192402819058759 


Optimalen k s CV

In [5]:
k_range = list(range(1, 100))
knn = KNeighborsRegressor()
param_grid = dict(n_neighbors=k_range)
# grid with 10 folkd cross validation
grid = GridSearchCV(knn, param_grid,  cv=5, scoring='neg_mean_squared_error')
grid.fit(X_train, y_train)

print('Napaka: ', -grid.best_score_)
print('Optimalni parametri: ', grid.best_params_)

Napaka:  0.023355105347579144
Optimalni parametri:  {'n_neighbors': 4}


Linearna regresija + izbira nazaj

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFECV
estimator = LinearRegression()
selector = RFECV(estimator, step=1)
selector.fit(X_train, y_train)
print('ranking: ', selector.ranking_)
print('selected features: ', selector.get_support(indices=True))

ranking:  [1 1 2 3 1]
selected features:  [0 1 4]


Zanesljivost izbire

In [8]:
reg = LinearRegression()
reg.fit(X_train, y_train)
err_cel_X = mean_squared_error(y_test, reg.predict(X_test))
err_fs = mean_squared_error(y_test, selector.predict(X_test))
print("Napaka na celi množici: ", err_cel_X)
print("Napaka, če upoštevamo le izbrane značilke: ", err_fs)

Napaka na celi množici:  0.11828449048167765
Napaka, če upoštevamo le izbrane značilke:  0.11799456316147854


Ocena se zdi zelo zanesljiva, saj sta napaki skoraj enaki.